In [1]:
from pathlib import Path
from collections import defaultdict

In [55]:
t = Path('day07.txt').read_text().splitlines()

In [59]:
t = """$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k""".splitlines()

In [3]:
def serve(x):
    c = None
    r = []
    for line in x:
        if line.startswith('$'):
            if c is not None:
                yield c, r
            c = line[2:]
            r = []
        else:
            r.append(line)
    else: 
        yield c, r            
        
        
        
        

In [4]:
list(serve(t[1:]))

[('ls', ['dir a', '14848514 b.txt', '8504156 c.dat', 'dir d']),
 ('cd a', []),
 ('ls', ['dir e', '29116 f', '2557 g', '62596 h.lst']),
 ('cd e', []),
 ('ls', ['584 i']),
 ('cd ..', []),
 ('cd ..', []),
 ('cd d', []),
 ('ls', ['4060174 j', '8033020 d.log', '5626152 d.ext', '7214296 k'])]

In [61]:
# create filesystem
dir_cnt = 1
fs = {1 : ("dir", "/", 0, [])}
current_dir_id = 1

for cmd, content in list(serve(t[1:])):
    
    if cmd == 'ls':
        for x in content:
            ft, name = x.split(' ')
            # print(current_dir_id, cmd, '->', content, ft, name)
            if ft == 'dir':
                dir_cnt += 1
                fs[dir_cnt] = ("dir", name, current_dir_id, [])
                fs[current_dir_id][3].append(('dir', dir_cnt, name)) 
            else:                  
                fs[current_dir_id][3].append(('file', int(ft), name)) 
    elif cmd == 'cd ..':                
        current_dir_id = fs[current_dir_id][2]
    elif cmd.startswith('cd '): 
        name = cmd[3:]
        current_dir_id = [x[1] for x in fs[current_dir_id][3] if x[0] == 'dir' and x[2] == name][0]


In [62]:
fs

{1: ('dir',
  '/',
  0,
  [('dir', 2, 'a'),
   ('file', 14848514, 'b.txt'),
   ('file', 8504156, 'c.dat'),
   ('dir', 3, 'd')]),
 2: ('dir',
  'a',
  1,
  [('dir', 4, 'e'),
   ('file', 29116, 'f'),
   ('file', 2557, 'g'),
   ('file', 62596, 'h.lst')]),
 3: ('dir',
  'd',
  1,
  [('file', 4060174, 'j'),
   ('file', 8033020, 'd.log'),
   ('file', 5626152, 'd.ext'),
   ('file', 7214296, 'k')]),
 4: ('dir', 'e', 2, [('file', 584, 'i')])}

In [45]:
def get_total_dir_size(id):
    total = 0
    for item in fs[id][3]:
        if item[0] == 'file':
            total += item[1]
        elif item[0] == 'dir':     
            total += get_total_dir_size(item[1])
        else:
            raise RuntimeError(str(item))            
            
    
    return total



In [57]:
sum([v for v in [get_total_dir_size(k) for k in fs ] if v <= 100_000])

1845346

In [58]:
# part 2

In [71]:
used_space = get_total_dir_size(1) 

In [73]:
free_space = 70_000_000 - used_space

In [75]:
required_space = 30000000 - free_space

In [76]:
required_space

8381165

In [ ]:
[v for v in [get_total_dir_size(k) for k in fs ] if v <= 100_000]